In [1]:
from vivit import run_experiment, test_files, classes
from data import get_video_label, VideoDataGenerator, labels_df, files, labels
import synthetic
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
classes = list(labels_df.groupby('label').size().sort_values(ascending=False)[:10].index)
good_classes = ['book', "drink", "go", "chair", "who", "clothes", "yes", "year", "thin", "fine"]
test_files = [f for f in files if labels[int(os.path.basename(f).split('.')[0])] in good_classes]

In [3]:
base = "../data/videos"
for video, f in enumerate(test_files):
    name = os.path.basename(f)
    if not os.path.exists(f"{base}/flipped/{name}"):
        synthetic.flip_video(f"{base}/original/{name}", f"{base}/flipped/{name}")
    if not os.path.exists(f"{base}/rotated_90/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_90/{name}", 90)
    if not os.path.exists(f"{base}/rotated_270/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_270/{name}", 270)
    if not os.path.exists(f"{base}/noised/{name}"):
        synthetic.noise_video(f"{base}/original/{name}", f"{base}/noised/{name}")
    print(f"{video+1}/{len(test_files)}", end = '\r')

In [4]:
all_files = []
for folder in ["original", "flipped", "rotated_90", "rotated_270", "noised"]:
    all_files += [f"{base}/{folder}/{os.path.basename(f)}" for f in test_files]
y = [get_video_label(f) for f in all_files]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(all_files, y, test_size=0.2, random_state=42, stratify=y)
train, validation = VideoDataGenerator(X_train), VideoDataGenerator(X_test)

In [6]:
model = run_experiment(data = train, validation = validation, epochs = 100)

2024-04-16 11:32:57.434169: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-16 11:32:57.434189: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-16 11:32:57.434192: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-16 11:32:57.434208: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-16 11:32:57.434220: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-04-16 11:33:04.509777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x2ff0f3a50] moov atom not found
OpenCV: Couldn't read video stream from file "../data/videos/rotated_270/57944.mp4"
2024-04-16 11:33:12.099541: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 1 dimension(s)
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  F

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 1 dimension(s)
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generat

In [ ]:
model.save("model-small-patch-time.keras")